# Samuel Watkins, 3032132676

# Interaction with the World Homework (#3)
Python Computing for Data Science (c) J Bloom, UC Berkeley 2018

Due Tuesday 2pm, Feb 20, 2018

# 1) Monty: The Python Siri

Let's make a Siri-like program (call it Monty!) with the following properties:
   - record your voice command
   - use a webservice to parse that sound file into text
   - based on what the text, take three different types of actions:
       - send an email to yourself
       - do some math
       - tell a joke

So for example, if you say "Monty: email me with subject hello and body goodbye", it will email you with the appropriate subject and body. If you say "Monty: tell me a joke" then it will go to the web and find a joke and print it for you. If you say, "Monty: calculate two times three" it should response with printing the number 6.

Hint: you can use speed-to-text apps like Houndify (or, e.g., Google Speech https://cloud.google.com/speech/) to return the text (but not do the actions). You'll need to sign up for a free API and then follow documentation instructions for using the service within Python. 

In [44]:
import pyaudio
import smtplib
import os
import my_credentials as creds
import wave
import houndify
import numpy as np

from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email import encoders
from email.utils import COMMASPACE, formatdate

In [47]:
# Houndify listener, taken from Houndify SDK sample_wave.py
class MyListener(houndify.HoundListener):
    def onPartialTranscript(self, transcript):
        return
    def onFinalResponse(self, response):
        textString=response['AllResults'][0]['WrittenResponseLong']
        print(f"Monty heard: \"{textString}\"")
        return
    def onError(self, err):
        print("Error: " + str(err))

# taken from lecture, changed print outputs
def MontyListens(RECORD_SECONDS=5):
    # can change number of seconds to record by specifying RECORD_SECONDS
    # set up pyaudio for recording
    p = pyaudio.PyAudio()
    
    chunk = 1024
    FORMAT = p.get_format_from_width(2) # Houndify requires a sample width of 2
    CHANNELS = 1
    RATE = 16000
    WAVE_OUTPUT_FILENAME = "MontySnippet.wav"
    
    stream = p.open(format = FORMAT,
        channels = CHANNELS,
        rate = RATE,
        input = True,
        frames_per_buffer = chunk)
    print("Monty is listening!")
    
    # record data
    all = []
    for i in range(0, int(RATE / chunk * RECORD_SECONDS)):
        data = stream.read(chunk)
        all.append(data)
    print("Monty is thinking...")
    # stop listening and close pyaudio
    stream.close()
    p.terminate()
    
    # with data recorded, create a .wav file
    data = b"".join(all)
    wf = wave.open(WAVE_OUTPUT_FILENAME, "wb")
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()
    
    # return the .wav filename to open using Houndify
    return WAVE_OUTPUT_FILENAME

def MontyConverts(AUDIO_FILE,CLIENT_ID,CLIENT_KEY):
    # convert speech to text via Houndify
    # AUDIO_FILE = filename of the .wav file to listen to 
    # CLIENT_ID = Houndify API Client ID
    # CLIENT_KEY = Houndify API Client Key
    BUFFER_SIZE = 512
    
    # open .wav file, make sure it has correct attributes
    audio = wave.open(AUDIO_FILE)
    
    if audio.getsampwidth() != 2:
        print("%s: wrong sample width (must be 16-bit)" % AUDIO_FILE)
        audio.close()
        return
    if audio.getframerate() != 8000 and audio.getframerate() != 16000:
        print("%s: unsupported sampling frequency (must be either 8 or 16 khz)" % AUDIO_FILE)
        audio.close()
        return
    if audio.getnchannels() != 1:
        print("%s: must be single channel (mono)" % AUDIO_FILE)
        audio.close()
        return
    
    # set up client
    client = houndify.StreamingHoundClient(CLIENT_ID, CLIENT_KEY, "test_user")
    client.setLocation(37.388309, -121.973968)
    
    # listen to the .wav file
    client.setSampleRate(audio.getframerate())
    client.start(MyListener())

    while True:
        samples = audio.readframes(BUFFER_SIZE)
        if len(samples) == 0: 
            break
        if client.fill(samples): 
            break

    audio.close()
    result = client.finish() # returns either final response or error
    return result['AllResults'][0]['WrittenResponseLong'].lower() # return the response, all lowercase

# function that figures out what the command is
def MontyThinks(textResult):
    # textResult = string of Houndify output
    # returns 0 if an email, 1 if a math problem, 2 if a joke, and -1 if Monty doesn't know what to do
    
    textArray = np.array(textResult.split(" ")) # split into an array using space as delimiter
    
    # check for the key word
    if any(textArray=="email"):
        print("Monty is sending your email...")
        return 0
    elif any(textArray=="calculate"):
        print("Monty is calculating...")
        return 1
    elif any(textArray=="joke"):
        print("Monty is thinking of a joke...")
        return 2
    else:
        print("Monty doesn't know what to do... Sorry!")
        return -1
        

# from the email_example.py code, removed file attachment ability, SMTP -> SMTP_SSL
def MontyEmails(sender, pwd, to, text):
    # sender = email of sender
    # pwd = password of sender (if using 2-step authentication, create an app password)
    # to = email of recipient
    # text = what Monty heard
    
    textArray = np.array(text.split(" ")) # split into an array using space as delimiter
    numWords = len(textArray)
    # find where the subject and body start 
    subjectIndex = np.where(textArray=="subject")[0]
    bodyIndex = np.where(textArray=="body")[0]
    
    # check if body was said first or if subject was said first (assuming the first instance of the word)
    if subjectIndex[0] < bodyIndex[0]:
        if not bodyIndex:
            bodyStr = ["(No Body)"] # if body was not said
        else:
            bodyStr = textArray[bodyIndex[0]+1:]
        if not subjectIndex:
            subjectStr = ["(No Subject)"] # if subject was not said
        else:
            subjectStr = textArray[subjectIndex[0]+1:bodyIndex[0]]
    else:
        if not bodyIndex:
            bodyStr = ["(No Body)"] # if body was not said
        else:
            bodyStr = textArray[bodyIndex[0]+1:subjectIndex[0]]
        if not subjectIndex:
            subjectStr = ["(No Subject)"] # if subject was not said
        else:
            subjectStr = textArray[subjectIndex[0]+1:]
    
    # if the last word of a string is and, then don't send that part
    if len(subjectStr)>1 and subjectStr[-1]=="and":
        subjectStr = subjectStr[:-1]
    if len(bodyStr)>1 and bodyStr[-1]=="and":
        bodyStr = bodyStr[-1]
    
    # join the subject and body into one string, delimited by spaces
    subject = " ".join(subjectStr)
    body = " ".join(bodyStr)
    
    # create email message
    msg = MIMEMultipart()
    msg["From"] = sender
    msg["To"] = COMMASPACE.join(to)
    msg["Date"] = formatdate(localtime=True)
    msg["Subject"] = subject
    msg.attach(MIMEText(body))
    
    # open mail server, login, send message, close server
    mailServer = smtplib.SMTP_SSL("smtp.gmail.com", creds.GMAIL_SMTP_PORT)
    mailServer.login(sender, pwd)
    mailServer.sendmail(sender, to, msg.as_string())
    mailServer.close()
    
    print(f"Monty sent an email to {COMMASPACE.join(to)} with subject \"{subject}\" and body \"{body}\"")


In [48]:
# Load up the Gmail user, pass, and Houndify credentials
GMAIL_USER = creds.GMAIL_USERNAME
GMAIL_PASS = creds.GMAIL_PASSWORD
CLIENT_ID = creds.HOUNDIFY_CLIENT_ID
CLIENT_KEY = creds.HOUNDIFY_CLIENT_KEY

# Boot up Monty, listen to the user (default is for ten seconds)
wavFilename = MontyListens(RECORD_SECONDS=10)

textResult = MontyConverts(wavFilename,CLIENT_ID,CLIENT_KEY)

# textResult = "hey manti send me an email with subject hello body pillow"

# is it an email command, a math command, or a joke

resultCode = MontyThinks(textResult)

if resultCode == 0:
    MontyEmails(sender=GMAIL_USER, 
                pwd=GMAIL_PASS, 
                to=[GMAIL_USER,],   # include an extra comma in the "to" list to account for the COMMASPACE.join(to)
                text=textResult)


Monty is listening!
Monty is thinking...
Monty heard: "Hey manti send email subject hollow body what's up"
Monty is sending your email...
Monty sent an email to samwatkinss@gmail.com with subject "hollow" and body "what's up"


In [17]:
not np.where(np.array(textResult.split(" "))=="banana")[0]

True

# 2) Write a program that identifies musical notes from sound (AIFF) files. 

  - Run it on the supplied sound files (12) and report your program’s results. 
  - Use the labeled sounds (4) to make sure it works correctly. The provided sound files contain 1-3 simultaneous notes from different organs.
  - Save copies of any example plots to illustrate how your program works.
  
  https://piazza.com/berkeley/spring2018/ay250class13410/resources -> Homeworks -> hw3_sound_files.zip

Hints: You’ll want to decompose the sound into a frequency power spectrum. Use a Fast Fourier Transform. Be care about “unpacking” the string hexcode into python data structures. The sound files use 32 bit data. Play around with what happens when you convert the string data to other integer sizes, or signed vs unsigned integers. Also, beware of harmonics.